Basic setup for polars with s3fs.

In [9]:
import s3fs
import polars as pl
import json

s3 = s3fs.S3FileSystem()

I explicitly set the schema for productivity here, rather than infer, mostly to avoid the need to trim whitespace from 
column names.

In [10]:
productivity_schema = {
    "series_id": pl.Utf8,
    "year": pl.Int64,
    "period": pl.Utf8,
    "value": pl.Float64,
    "footnote_codes": pl.Utf8
}

Part 3.0: Load BLS productivity and datanation USA population datasets from s3. Trim whitespace from productivity. 
Get just the `data` element from the population json, as there was also metadata in that file.

In [11]:
productivity_df = pl.read_csv("s3://newell-data-quest/pr.data.0.Current", separator='\t',
                              schema=productivity_schema).with_columns(pl.col(pl.Utf8).str.strip_chars())
population_df = pl.DataFrame(json.loads(s3.open('s3://newell-data-quest/datausa_nation_pop.json').read())['data'])


Part 3.1: Filter USA population down to the required years, and calculate population mean.

In [12]:
selected_years_df = population_df.filter(
    pl.col('ID Year') >= 2013,
    pl.col('ID Year') <= 2018
)

selected_years_df.mean().select(pl.col('Population').alias('population_mean'))

population_mean
f64
3.17437383e8


And the standard deviation.

In [13]:
selected_years_df.std().select(pl.col('Population').alias('population_std'))

population_std
f64
4.2571e6


Part 3.2 Load BLS productivity from s3, find the "best year."

In [14]:
(productivity_df
    .group_by('series_id', 'year')
    .agg([
        pl.sum('value').alias('total_value'),
    ])
    .sort('series_id', 'total_value', descending=[False, True])
    .group_by('series_id')
    .first()
    .sort('series_id')
 )


series_id,year,total_value
str,i64,f64
"""PRS30006011""",2022,21.4
"""PRS30006012""",2022,17.6
"""PRS30006013""",1998,703.575
"""PRS30006021""",2010,17.7
"""PRS30006022""",2010,12.5
"""PRS30006023""",2014,502.951
"""PRS30006031""",2022,21.6
"""PRS30006032""",2021,17.6
"""PRS30006033""",1998,699.946


Part 3.3: Join the productivity and population dataframes, for series_id = PRS30006032 and period = Q01.

In [15]:
(population_df
    .join(productivity_df, left_on='ID Year', right_on='year', how='inner')
    .filter(
        pl.col('series_id') == 'PRS30006032',
        pl.col('period') == 'Q01'
    )
    .select('series_id', 'ID Year', 'period', 'value', 'Population'))

series_id,ID Year,period,value,Population
str,i64,str,f64,i64
"""PRS30006032""",2013,"""Q01""",1.2,311536594
"""PRS30006032""",2014,"""Q01""",0.0,314107084
"""PRS30006032""",2015,"""Q01""",-1.7,316515021
"""PRS30006032""",2016,"""Q01""",-1.8,318558162
"""PRS30006032""",2017,"""Q01""",1.0,321004407
"""PRS30006032""",2018,"""Q01""",-0.1,322903030
"""PRS30006032""",2019,"""Q01""",-2.4,324697795
"""PRS30006032""",2020,"""Q01""",-6.5,326569308
"""PRS30006032""",2021,"""Q01""",1.3,329725481
